# Installlations

In [ ]:
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric
#!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=2657108 sha256=026cf249e122f4c8aa38065fdabe9750cb7874a4cfb6cede1fb99c34f09a2a0d
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
     |████████████████████████████████| 2.1 MB 33.2 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=1577337 sha256=7656d0f3850953546ed49496b2e52f18921d13f46306fb7dc98d0a8668d23608
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 19.8 MB/s 
     |████████████████████████████████| 81 kB 5.9 MB/s 
     |████████████████████████████████| 209 kB 59.1 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 112 kB 68.0 MB/s 
     |████████████████████████████████| 147 kB 63.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=43b77773b4dab96c1e816e9c54fd02c42c0c282c1cb7fff1ede45dbfe3e68b29
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.0.0
    Uninstalling importlib-metadata-5.0.0:
      Successfully uninstalled importlib-metadata-5.0.0


# Model definitions

In [ ]:
import torch
from torch_geometric.datasets import UPFD
import numpy as np
from torch_geometric.loader import DataLoader

In [ ]:
import torch.nn.functional as F
from torch.nn import ReLU, LeakyReLU, Softmax, Linear, SELU, GELU, ELU
from torch_geometric.nn import SAGEConv, global_max_pool, TopKPooling,GCNConv

In [ ]:
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score 

In [ ]:
test_data_gos = UPFD(root=".", name="gossipcop", feature="bert",split="test")
train_data_gos = UPFD(root=".", name="gossipcop", feature="bert", split="train")
val_data_gos = UPFD(root=".", name="gossipcop", feature="bert", split="val")

In [ ]:
train_loader = DataLoader(train_data_gos, batch_size=32, shuffle=True)
test_loader = DataLoader(train_data_gos, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data_gos, batch_size=32, shuffle=True)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.leaky1 = LeakyReLU()
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.leaky2 = LeakyReLU()
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        self.leaky3 = LeakyReLU() 
        
        self.full1 = Linear(hidden_channels[2],hidden_channels[3])
        self.ge1 = GELU()
        self.full2 = Linear(hidden_channels[3],hidden_channels[4])
        self.ge2 = GELU()
        self.full3 = Linear(hidden_channels[4],out_channels)


    def forward(self, x, edge_index, batch):
        h = self.leaky1(self.conv1(x, edge_index))
        h = self.leaky2(self.conv2(h, edge_index))
        h = self.leaky3(self.conv3(h, edge_index))

        h = global_max_pool(h,batch)

        h = self.ge1(self.full1(h))
        h = self.ge2(self.full2(h))
        h = self.full3(h)

        return torch.sigmoid(h)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper Parameter Tuning

In [ ]:
def objective(trial):
  
  model = Net(768,[trial.suggest_int(name="layer_size1",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size2",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size3",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size4",low=64,high=256,step=64),
                   trial.suggest_int(name="layer_size5",low=64,high=256,step=64),
  
  ],1).to(device)
  optimizer = torch.optim.Adam(model.parameters(),lr=trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),betas=(trial.suggest_loguniform('b1', 1-1e-1,1-1e-3),0.99))
  lossff = torch.nn.BCELoss()

  total_loss = 0
  weighted_loss = 0
  exp_param = 0.8

  wloss = []

  for i in range(400):
    print("Epoch:", i)
    model.train()
    for data in train_loader:
          data = data.to(device)
          optimizer.zero_grad()
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          loss.backward()
          optimizer.step()
          total_loss += float(loss) * data.num_graphs
    print("Train: ",total_loss / len(train_loader.dataset))

    model.eval()
    total_loss = 0
    for data in val_loader:
          data = data.to(device)
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          total_loss += float(loss) * data.num_graphs
          
    print("Test", total_loss / len(val_loader.dataset))

    weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
    print(weighted_loss/(1-exp_param**(i+1)))
    wloss.append(weighted_loss/(1-exp_param**(i+1)))

    if(i-30>=0 and wloss[i-20]-weighted_loss<0.01):
      break

  return weighted_loss

In [ ]:
import optuna
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(),pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=30)

[I 2022-10-15 12:18:05,824] A new study created in memory with name: no-name-f44bd841-3251-4e67-a81c-665ab8c6cedc
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  # Remove the CWD from sys.path while we load stuff.


Epoch: 0
Train:  0.6734409181626289
Test 0.5883285658700126
0.5883285658700126
Epoch: 1
Train:  0.7410202604947073
Test 0.2711249816985357
0.4121043524414144
Epoch: 2
Train:  0.4425303549556942
Test 0.2491557567820444
0.34532214110560705
Epoch: 3
Train:  0.3476556937445651
Test 0.17865540948270003
0.28886322117914803
Epoch: 4
Train:  0.26342853388643306
Test 0.15664322930814584
0.2495307628643568
Epoch: 5
Train:  0.25067862810314573
Test 0.15549390199815943
0.22404154522129438
Epoch: 6
Train:  0.21862659711329327
Test 0.13647594966076232
0.2018810290224425
Epoch: 7
Train:  0.19661503752195464
Test 0.15832880553499465
0.19141461071866459
Epoch: 8
Train:  0.20783235950415935
Test 0.14639594572367687
0.18101507576607373
Epoch: 9
Train:  0.1809123311423775
Test 0.14106498194019218
0.1720639356493123
Epoch: 10
Train:  0.1753456720795769
Test 0.16153149421398455
0.16975949764495968
Epoch: 11
Train:  0.16975126131659463
Test 0.17073512066414942
0.16996902055343202
Epoch: 12
Train:  0.16623659

[I 2022-10-15 12:18:47,338] Trial 0 finished with value: 0.2897461642402153 and parameters: {'layer_size1': 512, 'layer_size2': 384, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 128, 'learning_rate': 0.00020517752953342884, 'b1': 0.9569164721605181}. Best is trial 0 with value: 0.2897461642402153.


Test 0.3413764289085627
0.2900333994064996
Epoch: 0
Train:  0.6118858931210888
Test 0.3116219233407642
0.3116219233407642
Epoch: 1
Train:  0.3718375306535553
Test 0.1469686738737337
0.22014789585908065
Epoch: 2
Train:  0.2260886481229639
Test 0.1330065532918378
0.1844342308725057
Epoch: 3
Train:  0.20478879867101107
Test 0.12764737277117733
0.16519749032327521
Epoch: 4
Train:  0.18214753935891356
Test 0.17781758365722802
0.1689516827715001
Epoch: 5
Train:  0.20255389864191467
Test 0.24854944521841002
0.1905271019445935
Epoch: 6
Train:  0.23827580191133604
Test 0.23491604708053254
0.20176075976920027
Epoch: 7
Train:  0.20604239176658593
Test 0.11720397661102343
0.18144017468563062
Epoch: 8
Train:  0.12935315598921351
Test 0.16030945084415948
0.17655887264818923
Epoch: 9
Train:  0.15393973502497643
Test 0.1252962908312685
0.16507307853240838
Epoch: 10
Train:  0.10629152996109921
Test 0.12505900568069528
0.15631822803030795
Epoch: 11
Train:  0.09695029641064941
Test 0.13859602965622622
0.

[I 2022-10-15 12:19:19,193] Trial 1 finished with value: 0.233908620886667 and parameters: {'layer_size1': 512, 'layer_size2': 256, 'layer_size3': 512, 'layer_size4': 64, 'layer_size5': 192, 'learning_rate': 0.0005269523924132061, 'b1': 0.9230938601001594}. Best is trial 1 with value: 0.233908620886667.


...

Train:  1.021026922669603
Test 0.6803910688602881
0.6834071817684076
Epoch: 0
Train:  0.6913126615377573
Test 0.6887525634451227
0.6887525634451226
Epoch: 1
Train:  1.0187225289397188
Test 0.6441585463045281
0.6639781094781259
Epoch: 2
Train:  0.8445633632359487
Test 0.28946695550457463
0.510489931620113
Epoch: 3
Train:  0.37580558974227624
Test 0.17015135626653174
0.3951990863106342
Epoch: 4
Train:  0.2787186721196541
Test 0.17645683906453868
0.3301282131412817
Epoch: 5
Train:  0.27423165029003505
Test 0.14795876444477737
0.28075016411911363
Epoch: 6
Train:  0.23478011908384908
Test 0.1481060723436403
0.24718148311310836
Epoch: 7
Train:  0.23091991571197798
Test 0.1418890518583221
0.22187773185797463
Epoch: 8
Train:  0.2153623287494366
Test 0.1411895997917805
0.20323837305479645
Epoch: 9
Train:  0.24472613354646788
Test 0.13570253338132585
0.18810642451782628
Epoch: 10
Train:  0.19554528265836693
Test 0.13145851466681932
0.17571218552496165
Epoch: 11
Train:  0.18911591761032523
Test 0

[I 2022-10-15 12:37:43,998] Trial 28 finished with value: 0.17100321042909944 and parameters: {'layer_size1': 256, 'layer_size2': 256, 'layer_size3': 512, 'layer_size4': 192, 'layer_size5': 256, 'learning_rate': 8.850407146076767e-05, 'b1': 0.9522436758621542}. Best is trial 3 with value: 0.12057808925100155.


Test 0.24412363977896412
0.171138800556129
Epoch: 0
Train:  0.6694128476219736
Test 0.6094488102001149
0.6094488102001149
Epoch: 1
Train:  0.6439275259882102
Test 0.1548087002757268
0.35687097135323265
Epoch: 2
Train:  0.2934605360454146
Test 0.171632180584001
0.28095343415272783
Epoch: 3
Train:  0.24928815291866988
Test 0.15355090605906951
0.23779539618062134
Epoch: 4
Train:  0.22604855878667518
Test 0.19224533012935094
0.22424528133909635
Epoch: 5
Train:  0.23520852679953033
Test 0.20555910631850527
0.21918028897728292
Epoch: 6
Train:  0.23739874608569092
Test 0.13668997862782234
0.19830419206906438
Epoch: 7
Train:  0.19382809904564322
Test 0.13473878794046112
0.18302823010927144
Epoch: 8
Train:  0.19437497610656115
Test 0.1343171643332029
0.17177573225815618
Epoch: 9
Train:  0.17470418341839924
Test 0.14162038779739058
0.16501918458360107
Epoch: 10
Train:  0.15295888573586286
Test 0.13994789683352132
0.15953373007961916
Epoch: 11
Train:  0.15278476649458647
Test 0.13005002131242127


[I 2022-10-15 12:38:08,464] Trial 29 finished with value: 0.23580519559443047 and parameters: {'layer_size1': 512, 'layer_size2': 256, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 128, 'learning_rate': 0.0002522877443320814, 'b1': 0.9241513045517684}. Best is trial 3 with value: 0.12057808925100155.


Test 0.262155719301168
0.2360389572552444


In [ ]:
best_trial = study.best_trial

In [ ]:
print(best_trial.params)

{'layer_size1': 384, 'layer_size2': 256, 'layer_size3': 512, 'layer_size4': 64, 'layer_size5': 192, 'learning_rate': 1.713644852336342e-05, 'b1': 0.9308401877808863}


In [ ]:
import json

In [ ]:
with open('mydata.json', 'w') as f:
    json.dump(best_trial.params, f)

# Training the Best Model

In [ ]:
model = Net(768,[384,256,512,64,192],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1.713644852336342e-05,betas=(0.9308401877808863,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(800):
  print("Epoch:", i)
  model.train()
  for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(train_loader.dataset))

  model.eval()
  total_loss = 0
  for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs
        
  print("Test", total_loss / len(val_loader.dataset))

  weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
  print(weighted_loss/(1-exp_param**(i+1)))
  wloss.append(weighted_loss/(1-exp_param**(i+1)))

  if(i-30>=0 and wloss[i-20]-weighted_loss<0.01):
    break

print(weighted_loss)

Epoch: 0
Train:  0.6929760463945158
Test 0.694690180348826
0.694690180348826
Epoch: 1
Train:  1.0394450098603636
Test 0.6933273186177125
0.6939330349426519
Epoch: 2
Train:  1.0374248870563159
Test 0.6918217061640142
0.6930677362628824
Epoch: 3
Train:  1.0343252289426195
Test 0.6882151238647572
0.6914238973746286
Epoch: 4
Train:  1.0275692784742558
Test 0.6811952276544256
0.6883810993855154
Epoch: 5
Train:  1.0133882542669554
Test 0.6654556358690227
0.6821670241414319
Epoch: 6
Train:  0.9830402219906832
Test 0.6342650241467542
0.6700443058518538
Epoch: 7
Train:  0.9238604623756129
Test 0.5714193637117798
0.6463428776131461
Epoch: 8
Train:  0.8145829250306
Test 0.4678690219536806
0.6051145315424459
Epoch: 9
Train:  0.6572714699056995
Test 0.3601686101365875
0.5502324260563098
Epoch: 10
Train:  0.5101345567094101
Test 0.2765192502475047
0.49034554716893286
Epoch: 11
Train:  0.40514286563837487
Test 0.22872637850897654
0.4341607216250635
Epoch: 12
Train:  0.3479664443067579
Test 0.19966006

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
@torch.no_grad()
def val(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(val_loader.dataset), accuracy, f1

@torch.no_grad()
def test(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [ ]:
val(model),test(model)

((0.11984186966122289, 0.9615384615384616, 0.9632224168126094),
 (0.07417334500195343, 0.9826007326007326, 0.9822926374650514))

In [ ]:
final = DataLoader(train_data_gos+val_data_gos, batch_size=32, shuffle=True)

In [ ]:
model = Net(768,[384,256,512,64,192],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1.713644852336342e-05,betas=(0.9308401877808863,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(70):
  print("Epoch:", i)
  model.train()
  total_loss = 0
  for data in final:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(final.dataset))

#   model.eval()
#   total_loss = 0
#   for data in test_loader:
#         data = data.to(device)
#         out = model(data.x, data.edge_index, data.batch)
#         loss = lossff(torch.reshape(out,(-1,)), data.y.float())
#         total_loss += float(loss) * data.num_graphs
        
#   print("Test", total_loss / len(test_loader.dataset))

#   weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(test_loader.dataset))
#   print(weighted_loss/(1-exp_param**(i+1)))
#   wloss.append(weighted_loss/(1-exp_param**(i+1)))

#   # if(i-30>=0 and wloss[i-20]-weighted_loss<0.001):
#   #   break

# print(weighted_loss)

Epoch: 0
Train:  0.693460147168319
Epoch: 1
Train:  0.691132366002261
Epoch: 2
Train:  0.685896981548477
Epoch: 3
Train:  0.6707335383027464
Epoch: 4
Train:  0.6309554183876122
Epoch: 5
Train:  0.5413306132777707
Epoch: 6
Train:  0.42419510654040743
Epoch: 7
Train:  0.3464714191189907
Epoch: 8
Train:  0.30392695237428713
Epoch: 9
Train:  0.27378853857153357
Epoch: 10
Train:  0.2462554309921969
Epoch: 11
Train:  0.2232214086603769
Epoch: 12
Train:  0.20418780991154858
Epoch: 13
Train:  0.18930831100332285
Epoch: 14
Train:  0.17810733654560187
Epoch: 15
Train:  0.17367749998712131
Epoch: 16
Train:  0.16784529633574433
Epoch: 17
Train:  0.16600180734790435
Epoch: 18
Train:  0.16978635965540587
Epoch: 19
Train:  0.15758961883984207
Epoch: 20
Train:  0.15486823184750018
Epoch: 21
Train:  0.15495840323604507
Epoch: 22
Train:  0.1526205661969307
Epoch: 23
Train:  0.14916721507633998
Epoch: 24
Train:  0.14761962139039722
Epoch: 25
Train:  0.14419375558330316
Epoch: 26
Train:  0.142109597215522

### Best Metrics found after training 
**Epoch: 114 |  TestLoss: 0.0640709913405106| TestAcc: 0.9862637362637363 | TestF1: 0.9859943977591035**

*Note : We choose the best epoch based on lowest Validation Loss.*
